In [2]:
# imports
import os
import pathlib
import pandas as pd
import cobra

In [5]:
# get data directory
current_dir = pathlib.Path(os.getcwd())
data_dir = current_dir.parent / "datasets"

In [6]:
# Read the first sheet of the protein measurement Excel file into a DataFrame
all_prot = pd.read_excel(data_dir / "A735_protein_data_CCM_PRM_raw_data.xlsx")

In [7]:
print(f"Number of measured Proteins: {all_prot["Protein Accession"].nunique()}")
all_prot.head()

Number of measured Proteins: 82


,Protein Accession,Protein,Peptide Sequence,Precursor Mz,Product Mz,Isotope Label Type,Total Area,BioReplicate,Condition
0,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,1991927.0,1.0,SQ
1,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2491982.0,2.0,SQ
2,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2275862.0,3.0,SQ
3,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2408610.0,1.0,Liver Mets.
4,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2042891.0,2.0,Liver Mets.


In [8]:
primary_prot = all_prot.loc[all_prot["Condition"] == "SQ"]

primary_prot.head(5)

,Protein Accession,Protein,Peptide Sequence,Precursor Mz,Product Mz,Isotope Label Type,Total Area,BioReplicate,Condition
0,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,1991927.0,1.0,SQ
1,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2491982.0,2.0,SQ
2,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,907.451973,light,2275862.0,3.0,SQ
15,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,792.425030,light,1991927.0,1.0,SQ
16,P52209,6PGD_HUMAN,VDDFLANEAK,561.277303,792.425030,light,2491982.0,2.0,SQ


In [9]:
primary_prot.loc[primary_prot["Isotope Label Type"] == "heavy", ["Total Area"]]

,Total Area
45,1.223529e+09
46,9.456593e+08
47,1.091832e+09
60,1.223529e+09
61,9.456593e+08
...,...
14229,2.441980e+08
14230,5.585936e+08
14243,2.855180e+08
14244,2.441980e+08


In [10]:
primary_prot.rename(columns={"Total Area": "Heavy Spike Area"}).loc[primary_prot["Isotope Label Type"] == "heavy", ["Heavy Spike Area"]].reset_index(drop=True)

,Heavy Spike Area
0,1.223529e+09
1,9.456593e+08
2,1.091832e+09
3,1.223529e+09
4,9.456593e+08
...,...
1417,2.441980e+08
1418,5.585936e+08
1419,2.855180e+08
1420,2.441980e+08


In [11]:
primary_comparison = pd.concat(
    [
        primary_prot.loc[primary_prot["Isotope Label Type"] == "light", ["Protein Accession", "Protein", "Peptide Sequence", "BioReplicate", "Total Area"]].reset_index(drop=True), 
        primary_prot.rename(columns={"Total Area": "Heavy Spike Area"}).loc[primary_prot["Isotope Label Type"] == "heavy", ["Heavy Spike Area"]].reset_index(drop=True)
    ], 
    axis=1
    )
primary_comparison.head()

,Protein Accession,Protein,Peptide Sequence,BioReplicate,Total Area,Heavy Spike Area
0,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09
1,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08
2,P52209,6PGD_HUMAN,VDDFLANEAK,3.0,2275862.0,1.091832e+09
3,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09
4,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08


In [12]:
primary_comparison["Relative Protein Content"] = primary_comparison["Total Area"] / primary_comparison["Heavy Spike Area"]
primary_comparison.head()

,Protein Accession,Protein,Peptide Sequence,BioReplicate,Total Area,Heavy Spike Area,Relative Protein Content
0,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09,0.001628
1,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08,0.002635
2,P52209,6PGD_HUMAN,VDDFLANEAK,3.0,2275862.0,1.091832e+09,0.002084
3,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09,0.001628
4,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08,0.002635


In [15]:
# sample weight: 10 ng = 10 * 10e-9 g =  0.00000001 g
# amount of heavy peptide: 50 fmol = 50 * 10e-12 mmol = 0.00000000005 mmol 
primary_comparison["Absolute Protein Content [mmol/gDW]"] = primary_comparison["Relative Protein Content"] * 0.00000000005 / 0.00000001
primary_comparison.head()


,Protein Accession,Protein,Peptide Sequence,BioReplicate,Total Area,Heavy Spike Area,Relative Protein Content,Absolute Protein Content [mmol/gDW]
0,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09,0.001628,0.000008
1,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08,0.002635,0.000013
2,P52209,6PGD_HUMAN,VDDFLANEAK,3.0,2275862.0,1.091832e+09,0.002084,0.000010
3,P52209,6PGD_HUMAN,VDDFLANEAK,1.0,1991927.0,1.223529e+09,0.001628,0.000008
4,P52209,6PGD_HUMAN,VDDFLANEAK,2.0,2491982.0,9.456593e+08,0.002635,0.000013


In [16]:
primary_comparison["Protein Accession"].nunique()

82

In [17]:
primary_conc = primary_comparison.groupby(["Protein Accession"])["Absolute Protein Content [mmol/gDW]"].agg(['mean', 'std']).reset_index()
primary_conc.head()

,Protein Accession,mean,std
0,O00330,0.000009,5.308544e-06
1,O00757,0.000004,2.432937e-06
2,O14556,0.000002,2.998146e-07
3,O43837,0.000008,1.889814e-06
4,O75390,0.000022,2.217726e-05


In [18]:
# extracting uniprot accession numbers from ihuman for easy access
ihuman = cobra.io.read_sbml_model(current_dir.parent / "models" / "Human-GEM.xml")

In [28]:
# retrieving protein accession numbers & gene ids
# and calculating number fpr synonyms
gene_id_mapping = {}
num_synonyms = 0
for gene in ihuman.genes:
    if isinstance(gene.annotation["uniprot"], list):
        for synonym in gene.annotation["uniprot"]:
            gene_id_mapping[synonym] = gene.id
            num_synonyms += 1
        num_synonyms -= 1 # to count only additional synonyms
    else:
        gene_id_mapping[gene.annotation["uniprot"]] = gene.id

print(f"Number of proteins (with protein accession) in Human-GEM: {len(gene_id_mapping.keys()) - num_synonyms}")
measured_proteins = primary_comparison["Protein Accession"].unique()
print(f"Number of proteins measuered: {len(measured_proteins)}")
measured_proteins_model = set(gene_id_mapping.keys()).intersection(set(measured_proteins))
print(f'Number of measured proteins present in the model: {len(measured_proteins_model)}')

Number of proteins (with protein accession) in Human-GEM: 2875
Number of proteins measuered: 82
Number of measured proteins present in the model: 80


In [29]:
gene_id_mapping

{'O60762': 'ENSG00000000419',
 'Q9BTY2': 'ENSG00000001036',
 'P48506': 'ENSG00000001084',
 'Q16850': 'ENSG00000001630',
 'P28838': 'ENSG00000002549',
 'O14792': 'ENSG00000002587',
 'P19801': 'ENSG00000002726',
 'Q76N89': 'ENSG00000002746',
 'Q9NR63': 'ENSG00000003137',
 'Q9Y216': 'ENSG00000003987',
 'P52569': 'ENSG00000003989',
 'P54819': 'ENSG00000004455',
 'P28907': 'ENSG00000004468',
 'Q02790': 'ENSG00000004478',
 'O14561': 'ENSG00000004779',
 'Q16654': 'ENSG00000004799',
 'Q86VW1': 'ENSG00000004809',
 'Q9UJS0': 'ENSG00000004864',
 'P02730': 'ENSG00000004939',
 'P53701': 'ENSG00000004961',
 'P05141': 'ENSG00000005022',
 'P52435': 'ENSG00000005075',
 'Q53FZ2': 'ENSG00000005187',
 'Q92793': 'ENSG00000005339',
 'P05164': 'ENSG00000005381',
 'P27169': 'ENSG00000005421',
 'Q9UKG9': 'ENSG00000005469',
 'P21439': 'ENSG00000005471',
 'O75592': 'ENSG00000005810',
 'Q15119': 'ENSG00000005882',
 'Q9NZC3': 'ENSG00000006007',
 'Q09428': 'ENSG00000006071',
 'Q53H12': 'ENSG00000006530',
 'P43353':

In [ ]:
# load A375 model to filter for genes in model
A375_ftINIT = cobra.io.read_sbml_model(current_dir.parent / "models" / "A375_ftINIT_model_prep.xml")
A375_model_genes = []
for gene in A375_ftINIT.genes:
    A375_model_genes.append(gene.id)
print(A375_model_genes)
print(len(A375_model_genes))

In [35]:
model_conc = primary_conc.loc[primary_conc["Protein Accession"].isin(measured_proteins_model)]
model_conc["Gene ID"] = [gene_id_mapping[accession_number] for accession_number in model_conc["Protein Accession"]]
print(len(model_conc))
final_conc = model_conc.loc[model_conc["Gene ID"].isin(A375_model_genes)]
print(len(final_conc))
final_conc.to_csv(data_dir / "A735_protein_data_prep.tsv", sep="\t", index=False, float_format="%.15g")
final_conc.head()

80
79


C:\Users\phili\AppData\Local\Temp\ipykernel_5464\3259074756.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_conc["Gene ID"] = [gene_id_mapping[accession_number] for accession_number in model_conc["Protein Accession"]]


,Protein Accession,mean,std,Gene ID
0,O00330,0.000009,5.308544e-06,ENSG00000110435
1,O00757,0.000004,2.432937e-06,ENSG00000130957
2,O14556,0.000002,2.998146e-07,ENSG00000105679
3,O43837,0.000008,1.889814e-06,ENSG00000101365
4,O75390,0.000022,2.217726e-05,ENSG00000062485
